In [1]:
import pandas as pd

In [2]:
light_levels=pd.read_json('light_levels.json')
light_levels.head(10)

,Date,Light Score
0,2010-09-26,4.0
1,2015-05-07,6.0
2,2012-03-07,NaN
3,2000-09-18,15.0
4,2000-11-10,15.0
5,2006-05-13,14.0
6,2010-05-03,4.0
7,2011-04-18,3.0
8,2007-10-22,15.0
9,2010-04-08,4.0


In [3]:
flight_call=pd.read_json('flight_call.json')
flight_call.head(10)

,Species,Family,Collisions,Flight,Call,Habitat,Stratum
0,Zonotrichia,albicollis,Passerellidae,10133,Yes,Forest,Lower
1,Junco,hyemalis,Passerellidae,6303,Yes,Edge,Lower
2,Melospiza,melodia,Passerellidae,5124,Yes,Edge,Lower
3,Melospiza,georgiana,Passerellidae,4910,Yes,Open,Lower
4,Seiurus,aurocapilla,Parulidae,4580,Yes,Forest,Lower
5,Catharus,guttatus,Turdidae,3729,Yes,Forest,Lower
6,Certhia,americana,Certhiidae,2676,Yes,Forest,Upper
7,Oreothlypis,peregrina,Parulidae,2515,Yes,Edge,Upper
8,Passerella,iliaca,Passerellidae,2443,Yes,Edge,Lower
9,catharus,ustulatus,Turdidae,2331,Yes,Forest,Lower


In [4]:
collisions_data = pd.read_json('chicago_collision_data.json')
collisions_data.head(10)

,Genus,Species,Date,Locality
0,Ammodramus,nelsoni,1982-10-03,MP
1,Ammodramus,nelsoni,1984-05-21,CHI
2,Ammodramus,nelsoni,1984-05-25,MP
3,Ammodramus,nelsoni,1985-10-08,MP
4,Ammodramus,nelsoni,1986-09-10,MP
5,Ammodramus,nelsoni,1986-09-12,MP
6,Ammodramus,nelsoni,1986-09-12,MP
7,Ammodramus,nelsoni,1986-09-13,MP
8,Ammodramus,nelsoni,1986-09-25,MP
9,Ammodramus,nelsoni,1987-09-29,MP
